In [ ]:
import uproot
import numpy as np
import pkgutil
import uproot_methods
import matplotlib as mpl
import matplotlib.pyplot as plt
import max_functions as mf
%matplotlib inline
import mplhep as hep
plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS
import seaborn as sns
from scipy.interpolate import griddata

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0.root")
fileRCR["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].keys()
dict(fileRCR["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].classes())
histRCR = fileRCR["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check/MFPAD3D_ALLdir_usr_gated"]

fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CR_newEfield_ALLCH_MFPAD_30_t0.root")
fileRCR["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].keys()
dict(fileRCR["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].classes())
histRCR = fileRCR["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check/MFPAD3D_ALLdir_usr_gated"]

fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0.root")
fileRCL["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].keys()
dict(fileRCL["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].classes())
histRCL = fileRCL["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check/MFPAD3D_ALLdir_usr_gated"]

fileSCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\S-C3H3F3O_550eV_CL_newEfield_ALLCH_MFPAD_30_t0.root")
fileSCL["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].keys()
dict(fileSCL["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check"].classes())
histSCL = fileSCL["angular_distr_el/ALL_CH/test/MFPADs_e_multinew_check/MFPAD3D_ALLdir_usr_gated"]

In [ ]:
phistRCR = histRCR.pandas()
nhistRCR = histRCR.numpy()
r_norm_nhistRCR = nhistRCR[0] / nhistRCR[0].sum(axis=1)[:,np.newaxis] # r values normalise to 1
phi_nhistRCR = nhistRCR[1][0][0] # phi
costheta_nhistRCR = nhistRCR[1][0][1] # cos(theta)

phistRCL = histRCL.pandas()
nhistRCL = histRCL.numpy()
r_norm_nhistRCL = nhistRCL[0] / nhistRCL[0].sum(axis=1)[:,np.newaxis] # z values normalise to 1
phi_nhistRCL = nhistRCL[1][0][0] # phi
costheta_nhistRCL = nhistRCL[1][0][1] # cos(theta)

phistSCR = histRCR.pandas()
nhistSCR = histRCR.numpy()
r_norm_nhistSCR = nhistSCR[0] / nhistRCR[0].sum(axis=1)[:,np.newaxis] # z values normalise to 1
phi_nhistSCR = nhistSCR[1][0][0] # phi
costheta_nhistSCR = nhistSCR[1][0][1] # cos(theta)

phistSCL = histSCL.pandas()
nhistSCL = histSCL.numpy()
r_norm_nhistSCL = nhistSCL[0] / nhistSCL[0].sum(axis=1)[:,np.newaxis] # z values normalise to 1
phi_nhistSCL = nhistSCL[1][0][0] # phi
costheta_nhistSCL = nhistSCL[1][0][1] # cos(theta)

totR = np.add(r_norm_nhistRCL,r_norm_nhistRCR)
totS = np.add(r_norm_nhistSCL,r_norm_nhistSCR)

PECD =  np.divide(np.subtract(totR,totS),np.add(totR,totS))

In [ ]:
nhistRCL[0].shape

In [ ]:
(nhistRCL[0]/nhistRCL[0].sum(axis=1)[:,np.newaxis]).shape

In [ ]:
print(r_norm_nhistRCR.shape)
print(nhistRCR[0].shape)
print(phi_nhistRCR.shape)
print(costheta_nhistRCR.shape)
# x=r_norm_nhistRCL*np.sin(np.arccos(costheta_nhistRCL))*np.cos(phi_nhistRCL)
# y=r_norm_nhistRCL*np.sin(np.arccos(costheta_nhistRCL))*np.sin(phi_nhistRCL)
# z=r_norm_nhistRCL*costheta_nhistRCL

In [ ]:
cs = plt.contourf(PECD.T, 10, extent= [costheta_nhistSCL[0],costheta_nhistSCL[-1],phi_nhistSCL[0],phi_nhistSCL[-1]], aspect="auto", cmap='seismic') # first option with increasing number of levels = 10
cbar = plt.colorbar(cs)
plt.title("Contrast map with the right axis, 10 levels")
plt.show()

cs1 = plt.contourf(PECD.T, 8, extent= [costheta_nhistSCL[0],costheta_nhistSCL[-1],phi_nhistSCL[0],phi_nhistSCL[-1]], aspect="equal",cmap='seismic') # first option with increasing number of levels
cbar = plt.colorbar(cs1)
plt.title("Contrast map with the right axis, 8 levels")
plt.show()

In [ ]:
plt.imshow(PECD.T, extent= [costheta_nhistSCL[0],costheta_nhistSCL[-1],phi_nhistSCL[0],phi_nhistSCL[-1]], interpolation='quadric', cmap='seismic', aspect="auto")
# interpolation: [bicubic, quadric, spline16]
plt.colorbar()
# plt.axis(aspect='image');

In [ ]:
import matplotlib as mpl
from matplotlib import cm

magma_cmap = mpl.cm.get_cmap('magma')
seismic_cmap = mpl.cm.get_cmap('seismic')

magma_rgb = []
seismic_rgb = []
norm = mpl.colors.Normalize(vmin=0, vmax=255)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(magma_cmap(norm(i)))
       magma_rgb.append(k)

for i in range(0, 255):
       k = mpl.colors.colorConverter.to_rgb(seismic_cmap(norm(i)))
       seismic_rgb.append(k)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

Magma_r = matplotlib_to_plotly(magma_cmap, 255)
Seismic_r = matplotlib_to_plotly(seismic_cmap, 255)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure(data=go.Contour(z=PECD.T, x=costheta_nhistSCL, y=phi_nhistSCL, line_smoothing=0.85, colorscale=Seismic_r, contours=dict(start=-1, end=1, size=0.5) ))

fig.show()